In [1]:
from IPython.display import display
from cobramod import __version__
from cobra import __version__ as cobra_version
print(f'CobraMod version: {__version__}')
print(f'COBRApy version: {cobra_version}')
# From Escher:
# This option turns off the warning message if you leave or refresh this page
import escher
escher.rc['never_ask_before_quit'] = True

CobraMod version: 1.1.2-dev
COBRApy version: 0.29.0



# Analyising shikimate production using an *E. coli* core model

## Introduction

In this tutorial we use a core model of [*E. coli*](
http://bigg.ucsd.edu/models/e_coli_core
) [1] and extend it with the [shikimate synthesis pathway](
  https://biocyc.org/ECOLI/NEW-IMAGE?type=PATHWAY&object=ARO-PWY
). Shikimate is an important precursor for aromatic compounds such as
phenylalanine, tyrosine, and tryptophan. In [this article](
https://doi.org/10.1016/j.biortech.2014.05.035) the authors *in vivo*
engineered five *E. coli* strains for shikimate overproduction by deactivating
genes for the enzymatic regulation of the shikimate metabolism [2].
In our test case, we *in silico* compare the control strain (`W3110`) with a strain (`SA3`) from the article. To this end, we use CobraMod to add the shikimate pathway from the BioCyc
sub-database [EcoCyc](
 https://www.ecocyc.org/
) to the *E. coli* model. We then simulate and
visualize shikimate production for the two strains. The shikimate pathway is shown below and is represented by
two sub-pathways with the EcoCyc identifiers [PWY-6164](
  https://biocyc.org/ECOLI/NEW-IMAGE?type=PATHWAY&object=PWY-6164
) and [PWY-6163](
  https://biocyc.org/ECOLI/NEW-IMAGE?type=PATHWAY&object=PWY-6163
). 

<img src="https://websvc.biocyc.org/ECOLI/diagram-only?type=PATHWAY&object=ARO-PWY&pfontsize=normal&linear=snake"/>


<table style="border-collapse: collapse; width: 100%;" border="1">
<tbody>
<tr>
<td style="width: 33.3333%;"><strong>Pathway identifier</strong></td>
<td style="width: 33.3333%;"><strong>Pathway name</strong></td>
</tr>
<tr>
<td style="width: 33.3333%;">PWY-6164</td>
<td style="width: 33.3333%;">3-dehydroquinate biosynthesis I</td>
</tr>
<tr>
<td style="width: 33.3333%;">PWY-6163</td>
<td style="width: 33.3333%;">chorismate biosynthesis from 3-dehydroquinate</td>
</tr>
</tbody>
</table>




## Loading CobraMod, the *E. coli* model and setting the growth medium for the simulations

We use the *E. coli* core model from COBRApy as our model. This core model can be found under `cobramod.test.textbook`. It contains 72
metabolites, 95 reactions and 137 genes. We load the function
[cobramod.add_pathway()](
module/cobramod/index.html#cobramod.add_pathway
) and the Python module [pathlib](
https://docs.python.org/3/library/pathlib.html
) to represent the system path where CobraMod stores the metabolic pathway
information. We define the directory for pathway information as `dir_data`. The
pathways with the respective reaction, gene and metabolite information are 
saved here.

In [2]:
from pathlib import Path

from cobramod import add_pathway
from cobramod.test import textbook

# Defining system path for data
dir_data = Path.cwd().joinpath("data")
# Defining model
model = textbook.copy()

In the selected study, the authors used a culture medium including glucose and several other metabolites. Since only glucose is included in our core model and for the sake of simplicity we only constrain glucose and oxygen uptake.

In [3]:
# limiting glucose
model.exchanges.EX_glc__D_e.bounds = ( -10, 0)
# limiting oxygen
model.exchanges.EX_o2_e.bounds = (0, 0)

## Adding the shikimate pathways

To add a pathway to the model we use the function [add_pathway()](
module/cobramod/index.html#cobramod.add_pathway
). To do so, the first argument `model` is the respective model to extend. The
argument `pathway` is the respective database-specific identifier for the pathway (`PWY-6164` and `PWY-6163`) followed by the database identifier `ECOLI`.
We set cytosol (`c`) as the compartment for the reactions and metabolites
of the pathways. Because the two added sub-pathways are part of a larger pathway, we use
the argument `group` to create a single [cobramod.Pathway](
module/cobramod/index.html#cobramod.Pathway
) object.

CobraMod tries to avoid duplicate reactions and metabolites. In our case, the
`textbook` model uses BiGG identifiers and the downloaded data contain EcoCyc identifiers. When downloading pathway information, CobraMod checks the
cross-references within these data and searches for corresponding objects in the model. In this example CobraMod warns the user that several metabolites, 
such as `Pi` (phosphate), `WATER` and `ATP`, are already in the model. Calling the pathway object shows a table summarising the pathway attributes.

In [4]:
# first pathway
add_pathway(
    model=model,
    pathway="PWY-6164",
    database="ECOLI",
    compartment="c",
    group="PWY-SHIKIMATE",
    directory=dir_data,
)

# second pathway
add_pathway(
    model=model,
    pathway="PWY-6163",
    database="ECOLI",
    compartment="c",
    group="PWY-SHIKIMATE",
    directory=dir_data,
)

# Show attributes
model.groups.get_by_id("PWY-SHIKIMATE")

Number of       new   | removed entities in
*=====================|===================*
Reactions        2    |    0              
Metabolites      2    |    0              
Exchange         0    |    0              
Demand           0    |    0              
Sinks            2    |    0              
Genes            4    |    0              
Groups           1    |    0              

Number of       new   | removed entities in
*=====================|===================*
Reactions        5    |    0              
Metabolites      5    |    0              
Exchange         0    |    0              
Demand           0    |    0              
Sinks            2    |    1              
Genes            6    |    0              
Groups           0    |    0              



/home/stefano/work/projects/cobramod/src/cobramod/core/extension.py:211: UserWarning: The model cannot turnover the following metabolites ['3_DEOXY_D_ARABINO_HEPTULOSONATE_7_P_c']. To overcome this, sink reactions were created to simulate their synthesis.
  warn(msg)
/home/stefano/work/projects/cobramod/src/cobramod/utils.py:354: UserWarning: Auxiliary sink reaction for "SK_DEHYDROQUINATE_c" created. Consider removing it and adding the synthesis reactions for the metabolite.
  warn(message=msg, category=UserWarning)
/home/stefano/work/projects/cobramod/src/cobramod/utils.py:354: UserWarning: Auxiliary sink reaction for "SK_3_DEOXY_D_ARABINO_HEPTULOSONATE_7_P_c" created. Consider removing it and adding the synthesis reactions for the metabolite.
  warn(message=msg, category=UserWarning)
/home/stefano/work/projects/cobramod/src/cobramod/core/extension.py:211: UserWarning: The model cannot turnover the following metabolites ['3_DEHYDRO_SHIKIMATE_c']. To overcome this, sink reactions were 

Pathway identifier,PWY-SHIKIMATE
Name,
Memory address,0x0137113100185376
Reactions involved,"3_DEHYDROQUINATE_DEHYDRATASE_RXN_c, SHIKIMATE_5_DEHYDROGENASE_RXN_c, SHIKIMATE_KINASE_RXN_c, DAHPSYN_RXN_c, 2.5.1.19_RXN_c, CHORISMATE_SYNTHASE_RXN_c, 3_DEHYDROQUINATE_SYNTHASE_RXN_c"
Genes involved,"EG10076, EG10077, EG10082, EG10081, EG10079, EG10080, EG10078, EG10073, EG10075, EG10074"
Visualization attributes,vertical = False color_negative = None color_positive = None color_quantile = False


CobraMod does not automatically merge sub-pathways in the visualization. When we call
the function [Pathway.visualize()](
module/cobramod/core/pathway/index.html#cobramod.core.pathway.Pathway.visualize
), CobraMod shows both sub-pathways one above the other, instead of a single linear pathway.
To change this, we use the method [Pathway.modify_graph()](
module/cobramod/core/pathway/index.html#cobramod.core.pathway.Pathway.modify_graph
). This function merges the two sub-pathways by connecting two reactions in the 
visualization. Here, the argument `reaction` specifies the last reaction of the first sub-pathway and the argument `next_reaction` specifies the subsequent reaction of the second sub-pathway.

In the first visualization, `3_DEHYDROQUINATE_SYNTHASE_RXN_c` and
`3_DEHYDROQUINATE_DEHYDRATASE_RXN_c` are not connected. We use
the `modify_graph` method to join both reactions. When we now
visualize the pathway, we obtain one linear pathway.

In [5]:
# Show pathway map
display(model.groups.get_by_id("PWY-SHIKIMATE").visualize())
# Merge reactions of the sub-pathways
model.groups.get_by_id("PWY-SHIKIMATE").modify_graph(
    reaction="3_DEHYDROQUINATE_SYNTHASE_RXN_c",
    next_reaction="3_DEHYDROQUINATE_DEHYDRATASE_RXN_c"
)
# Show pathway map
model.groups.get_by_id("PWY-SHIKIMATE").visualize()

Builder(never_ask_before_quit=True, reaction_scale={}, reaction_styles=['color', 'text'])

Builder(never_ask_before_quit=True, reaction_scale={}, reaction_styles=['color', 'text'])

## Creating biomass reactions for the two *E. coli* strains

Our reference [article](
https://doi.org/10.1016/j.biortech.2014.05.035) lists the following values for biomass an shikimate for the two strains of interest:

<table style="border-collapse: collapse; width: 100%;" border="1">
<tbody>
<tr>
<td style="width: 33.3333%;"><strong>Strains</strong></td>
<td style="width: 33.3333%;"><strong>Biomass (g/L)</strong></td>
<td style="width: 33.3333%;"><strong>Shikimate (mg/L)</strong></td>
</tr>
<tr>
<td style="width: 33.3333%;">W3110</td>
<td style="width: 33.3333%;">3.42 &plusmn; 0.26</td>
<td style="width: 33.3333%;">1.31 &plusmn; 0.12</td>
</tr>
<tr>
<td style="width: 33.3333%;">SA3</td>
<td style="width: 33.3333%;">5.24 &plusmn; 0.34</td>
<td style="width: 33.3333%;">417.20 &plusmn; 50.01</td>
</tr>
</tbody>
</table>

We use this data to update the biomass equation of our core model by including the contribution of shikimate to the overall biomass composition. For the sake of our tutorial we make some simplifying assumptions (e.g. the ratios of the other biomass components do not change, shikimate productions is coupled to growth) and use the following relationship:


In [6]:
# Calculate ratios for the strains
shikimate_ratio = {
    "W3110": (1.31 / 1000) / 3.42,
    "SA3": (417.20 / 1000) / 5.24
}
shikimate_ratio

{'W3110': 0.00038304093567251464, 'SA3': 0.07961832061068702}


We calculate the new coefficients for the biomass reactions which include
shikimate. The detailed calculations are listed in the
[supplementary information](#Supplementary-Information:-Approximation-of-biomass-compositions) at the
bottom of this page. We obtain the following new biomass equations:

In [10]:
W3110_df = pd.concat(
    [
        W3110_df_negative.loc[:, ["identifier", "new_coefficient"]],
        W3110_df_positive.loc[:, ["identifier", "new_coefficient"]],
    ]
)
W3110_df = W3110_df.reset_index(drop=True)
print("New coefficients for W3110 strain:")
display(W3110_df)

SA3_df = pd.concat(
    [
        SA3_df_negative.loc[:, ["identifier", "new_coefficient"]],
        SA3_df_positive.loc[:, ["identifier", "new_coefficient"]],
    ]
)
SA3_df = SA3_df.reset_index(drop=True)
print("New coefficients for SA3 strain:")
display(SA3_df)

New coefficients for W3110 strain:


,identifier,new_coefficient
0,3pg_c,-1.495427
1,accoa_c,-3.746364
2,atp_c,-59.78709
3,e4p_c,-0.360862
4,f6p_c,-0.070873
5,g3p_c,-0.128951
6,g6p_c,-0.204921
7,gln__L_c,-0.255602
8,glu__L_c,-4.939507
9,h2o_c,-59.78709


New coefficients for SA3 strain:


,identifier,new_coefficient
0,3pg_c,-1.376891
1,accoa_c,-3.449406
2,atp_c,-55.048028
3,e4p_c,-0.332258
4,f6p_c,-0.065255
5,g3p_c,-0.118729
6,g6p_c,-0.188678
7,gln__L_c,-0.235342
8,glu__L_c,-4.547974
9,h2o_c,-55.048028


Finally, we create a function that uses these value to generate the updated strain-specific biomass equations and add it to the model.

In [11]:
from cobra import Reaction


def create_biomass(df: pd.DataFrame, identifier: str):
    """
    Creates a Reaction from the reactant and product DataFrame with new
    coefficients.
    """
    reaction = Reaction(id=identifier)
    # Iterate over dataframe
    for index, row in df.iterrows():
        reaction.add_metabolites(
            {
                model.metabolites.get_by_id(row["identifier"]): row[
                    "new_coefficient"
                ]
            }
        )
    return reaction


biomass_W3110 = create_biomass(W3110_df, "W3110_biomass")

biomass_SA3 = create_biomass(SA3_df, "SA3_biomass")
# Show new reactions
display(biomass_W3110)
display(biomass_SA3)

# Add to model
model.add_reactions([biomass_W3110, biomass_SA3])

Reaction identifier,W3110_biomass
Name,
Memory address,0x7cb421eee650
Stoichiometry,1.495426970760234 3pg_c + 0.10655682044443099 SHIKIMATE_c + 3.746364439181286 accoa_c + 59.78709032163742 atp_c + 0.3608617222222222 e4p_c + 0.07087284239766083 f6p_c + 0.12895058771929824 g3p_c +... 1.495426970760234 3-Phospho-D-glycerate + 0.10655682044443099 shikimate + 3.746364439181286 Acetyl-CoA + 59.78709032163742 ATP + 0.3608617222222222 D-Erythrose 4-phosphate + 0.07087284239766083...
GPR,
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,SA3_biomass
Name,
Memory address,0x7cb421eeea70
Stoichiometry,1.376890992366412 3pg_c + 22.148742610250682 SHIKIMATE_c + 3.4494064580152664 accoa_c + 55.0480282442748 atp_c + 0.33225778625954194 e4p_c + 0.06525506106870228 f6p_c + 0.11872923664122137 g3p_c +... 1.376890992366412 3-Phospho-D-glycerate + 22.148742610250682 shikimate + 3.4494064580152664 Acetyl-CoA + 55.0480282442748 ATP + 0.33225778625954194 D-Erythrose 4-phosphate + 0.06525506106870228...
GPR,
Lower bound,0.0
Upper bound,1000.0


## Simulating shikimate production in the two *E. coli* strains

We use COBRApy to run flux balance analysis for the two strains. Using the COBRApy method
[cobra.model.optimize()](
  https://cobrapy.readthedocs.io/en/latest/autoapi/cobra/core/model/index.html#cobra.core.model.Model.optimize
) we get two [solutions objects](
  https://cobrapy.readthedocs.io/en/latest/autoapi/cobra/core/solution/index.html#cobra.core.solution.Solution
). 
We can see that the solution for the W3110 strain has a higher objective value
(0.2105) than the SA3 strain objective value (0.0995) which is caused by the different shikimate ratios in the biomass equation.


In [12]:
# W3110 type
model.objective = "W3110_biomass"

solution_W3110 = model.optimize()
display(model.summary(solution=solution_W3110))

# SA3 strain
model.objective = "SA3_biomass"
solution_SA3 = model.optimize()
display(model.summary(solution=solution_SA3))

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,0.8114,1,0.01%
glc__D_e,EX_glc__D_e,10,6,0.60%
nh4_e,EX_nh4_e,7.177,0,0.00%
3_DEOXY_D_ARABINO_HEPTULOSONATE_7_P_c,SK_3_DEOXY_D_ARABINO_HEPTULOSONATE_7_P_c,0.1403,7,0.01%
3_ENOLPYRUVYL_SHIKIMATE_5P_c,SK_3_ENOLPYRUVYL_SHIKIMATE_5P_c,1000,10,99.39%
Metabolite,Reaction,Flux,C-Number,C-Flux
for_e,EX_for_e,-4.811,1,0.05%
h2o_e,EX_h2o_e,-8.772,0,0.00%
h_e,EX_h_e,-31.21,0,0.00%
pi_e,EX_pi_e,-995.3,0,0.00%


Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,0.60%
nh4_e,EX_nh4_e,6.023,0,0.00%
3_DEOXY_D_ARABINO_HEPTULOSONATE_7_P_c,SK_3_DEOXY_D_ARABINO_HEPTULOSONATE_7_P_c,26.58,7,1.86%
3_ENOLPYRUVYL_SHIKIMATE_5P_c,SK_3_ENOLPYRUVYL_SHIKIMATE_5P_c,977.5,10,97.54%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-14.96,1,0.15%
h2o_e,EX_h2o_e,-22.22,0,0.00%
h_e,EX_h_e,-21.37,0,0.00%
pi_e,EX_pi_e,-1000,0,0.00%
CHORISMATE_c,SK_CHORISMATE_c,-977.5,10,99.85%


## Visualization with Escher

The object [cobramod.Pathway](
module/cobramod/core/pathway/index.html#cobramod.core.pathway.Pathway
) has a method for visualizing metabolic pathway using the pathway visualization tool [Escher](
https://escher.readthedocs.io/en/latest/
).
The function
[Pathway.visualize()](
module/cobramod/core/pathway/index.html#cobramod.core.pathway.Pathway.visualize
) generates pathway maps which can be
easily customized with user-defined colors, flux ranges and gradient settings.

In [13]:
model.groups.get_by_id("PWY-SHIKIMATE").visualize()

Builder(never_ask_before_quit=True, reaction_scale={}, reaction_styles=['color', 'text'])

To visually compare our results we set the colors for positive and
negative fluxes to blue and red, respectively (By default, all color gradients use grey tones around zero, i.e. more saturated colors indicate larger absolute flux values). We set the map orientation to horizontal. To be able to compare the two flux simulations we set the bounds for the color gradient (`color_min_max`) to -3 and 3. The coloring of the arrows represents the values of the flux solution. To visualize the solution, we use the argument `solution_fluxes`.
The first visualization displays the flux distributions for the
W3110 strain. In this example, the flux values are close to 0 and thus the colors are pale. The second visualization displays the flux distributions for the SA3 strain which has higher fluxes and thus more saturated colors. Note that
the reaction `SHIKIMATE_5_DEHYDROGENASE_RXN_c` has a
negative flux because it is operating in the backwards direction ([reaction information](
  https://biocyc.org/ECOLI/NEW-IMAGE?type=REACTION&object=SHIKIMATE-5-DEHYDROGENASE-RXN
)).

In [14]:
# Customize pathway
model.groups.get_by_id("PWY-SHIKIMATE").vertical = False
model.groups.get_by_id("PWY-SHIKIMATE").color_positive = "blue"
model.groups.get_by_id("PWY-SHIKIMATE").color_negative = "red"
model.groups.get_by_id("PWY-SHIKIMATE").color_min_max = [-3, 3]

# Visualize pathway with solutions
display(model.groups.get_by_id("PWY-SHIKIMATE").visualize(
    solution_fluxes=solution_W3110)
)
display(model.groups.get_by_id("PWY-SHIKIMATE").visualize(
    solution_fluxes=solution_SA3)
)

Builder(never_ask_before_quit=True, reaction_data={'ACALD': 0.0, 'ACALDt': 0.0, 'ACKr': 0.0, 'ACONTa': 1.41796…

Builder(never_ask_before_quit=True, reaction_data={'ACALD': 0.0, 'ACALDt': 0.0, 'ACKr': 0.0, 'ACONTa': 0.79823…

## Supplementary Information: Approximation of biomass compositions

To add shikimate to the biomass reactions, we calculate the shikimate
contribution and the contribution of the other metabolite to biomass.

In [18]:
import pandas as pd

# Building dataframe from Biomass
biomass_df = pd.DataFrame(
    [metabolite.id, metabolite.formula, metabolite.formula_weight, value]
    for metabolite, value in model.reactions.get_by_id(
        "Biomass_Ecoli_core"
    ).metabolites.items()
)
biomass_df.columns = ["identifier", "formula", "mol_weight", "coefficient"]
biomass_df["mol_weight*coefficient"] = (
    biomass_df["mol_weight"] * biomass_df["coefficient"]
)
# display(biomass_df)

# Calculate Sum of coefficients multiplied by constants
sum_mol_coefficient_negative = biomass_df["mol_weight*coefficient"][
    biomass_df["mol_weight*coefficient"] < 0
].sum()
sum_mol_coefficient_positive = biomass_df["mol_weight*coefficient"][
    biomass_df["mol_weight*coefficient"] > 0
].sum()
print(
    "Sum of contribution for negative coefficients: "
    + f"{sum_mol_coefficient_negative}"
)
print(
    "Sum of contribution for positive coefficients: "
    + f"{sum_mol_coefficient_positive}"
)

Sum of contribution for negative coefficients: -48165.2120532678
Sum of contribution for positive coefficients: 46624.4458868131


We created a function that returns a table with the updated negative biomass coefficients. In the first part we calculate the contribution of
shikimate (`shikimate_mol_weight_coefficient`) to the biomass followed by the calculations for the remaining metabolites. Finally, we divide the values by the molecular weight of the respective metabolite which gives us the new biomass coefficients.

The two tables below show the new coefficients (`new_coefficient`) for the two 
strains.

In [19]:
def new_coefficient(ratio: float):
    """
    Returns the a dataframe with the new coefficient for reactants of the biomass
    equation.
    """
    SHIKIMATE_MOL_WEIGHT = 173.14
    shikimate_mol_weight_coefficient = sum_mol_coefficient_negative * ratio
    # Calculating new multiplication between the coefficients and constants
    new_mol_weight_coefficient = biomass_df["mol_weight*coefficient"][
        biomass_df["mol_weight*coefficient"] < 0
    ] * (1 - ratio)
    new_mol_weight_coefficient[
        len(new_mol_weight_coefficient)
    ] = shikimate_mol_weight_coefficient
    # Verify that the sums are the same
    assert round(new_mol_weight_coefficient.sum(), 4) == round(
        sum_mol_coefficient_negative, 4
    )

    # Creating series with names of metabolites
    identifiers = biomass_df["identifier"][
        biomass_df["mol_weight*coefficient"] < 0
    ]
    identifiers[len(identifiers)] = "SHIKIMATE_c"

    # Creating series with molecular weight of metabolites
    mol_weight = biomass_df["mol_weight"][
        biomass_df["mol_weight*coefficient"] < 0
    ]
    mol_weight[len(mol_weight)] = SHIKIMATE_MOL_WEIGHT

    # Building new dataframe
    new_coefficient_df = pd.DataFrame(
        [identifiers, new_mol_weight_coefficient]
    ).transpose()
    new_coefficient_df["new_coefficient"] = new_coefficient_df[
        "mol_weight*coefficient"
    ].divide(mol_weight)

    # Appending old coefficients
    new_coefficient_df["old_coefficient"] = biomass_df["coefficient"][
        biomass_df["mol_weight*coefficient"] < 0
    ]
    return new_coefficient_df

W3110_df_negative = new_coefficient(ratio=shikimate_ratio["W3110"])
SA3_df_negative = new_coefficient(ratio=shikimate_ratio["SA3"])

# Display and format only relevant information
print("Table with new negative coefficients for W3110 strain")
display(W3110_df_negative.loc[:,["identifier","new_coefficient"]])
print("Table with new negative coefficients for SA3 strain")
SA3_df_negative.loc[:,["identifier","new_coefficient"]]

Table with new negative coefficients for W3110 strain


,identifier,new_coefficient
0,3pg_c,-1.495427
1,accoa_c,-3.746364
2,atp_c,-59.78709
3,e4p_c,-0.360862
4,f6p_c,-0.070873
5,g3p_c,-0.128951
6,g6p_c,-0.204921
7,gln__L_c,-0.255602
8,glu__L_c,-4.939507
9,h2o_c,-59.78709


Table with new negative coefficients for SA3 strain


,identifier,new_coefficient
0,3pg_c,-1.376891
1,accoa_c,-3.449406
2,atp_c,-55.048028
3,e4p_c,-0.332258
4,f6p_c,-0.065255
5,g3p_c,-0.118729
6,g6p_c,-0.188678
7,gln__L_c,-0.235342
8,glu__L_c,-4.547974
9,h2o_c,-55.048028


Next, we calculate the updated positive coefficients for the biomass equations.
In the biomass equation are maintenance related metabolites such as `adp_c` and  `atp_c` for which we assign the absolute value of their respective counterpart. The only exception is `akg_c` and for the sake of simplicity for this tutorial, we use the same coefficient for this metabolite.

We create a function that returns a data frame with the new positive coefficient.
The tables below shows the updated positive coefficients for both strains.

In [20]:
def new_positive_coefficient(df: pd.DataFrame):
    """
    Returns a DataFrame with the new coefficients for the products of the
    biomass reaction. The input dataframe is the dataframe with the new
    coefficients for the reactants.
    """
    # Copy original biomass df and remove columns
    products = biomass_df[biomass_df["mol_weight*coefficient"] > 0].copy()
    del products["formula"]
    del products["mol_weight*coefficient"]
    products.columns = products.columns.str.replace(
        "coefficient", "old_coefficient"
    )   

    # Create new column with the equivalent new coefficients
    products["new_coefficient"] = [
        df["new_coefficient"][2],
        -4.1182,
        df["new_coefficient"][1],
        df["new_coefficient"][2],
        df["new_coefficient"][10],
        df["new_coefficient"][11],
        df["new_coefficient"][2],
    ]
    # Update index and convert to absolute value
    products.index = range(0, len(products["new_coefficient"]))
    products["new_coefficient"] = products["new_coefficient"] * -1
    return products

W3110_df_positive = new_positive_coefficient(W3110_df_negative)
SA3_df_positive = new_positive_coefficient(SA3_df_negative)

# Display and format only relevant information
print("Table with new positive coefficients for W3110 strain")
display(W3110_df_positive.loc[:,["identifier","new_coefficient"]])

print("Table with new positive coefficients for SA3 strain")
display(SA3_df_positive.loc[:,["identifier","new_coefficient"]])

Table with new positive coefficients for W3110 strain


,identifier,new_coefficient
0,adp_c,59.787090
1,akg_c,4.118200
2,coa_c,3.746364
3,h_c,59.787090
4,nadh_c,3.545641
5,nadp_c,13.022910
6,pi_c,59.787090


Table with new positive coefficients for SA3 strain


,identifier,new_coefficient
0,adp_c,55.048028
1,akg_c,4.118200
2,coa_c,3.449406
3,h_c,55.048028
4,nadh_c,3.264594
5,nadp_c,11.990640
6,pi_c,55.048028


## References

1. Orth, Jeffrey D., R. M. T. Fleming, and Bernhard Ø. Palsson. “Reconstruction and Use of Microbial Metabolic Networks: The Core Escherichia Coli Metabolic Model as an Educational Guide.” EcoSal Plus 4, no. 1 (February 1, 2010). https://doi.org/10.1128/ecosalplus.10.2.1.
2. Chen, Xianzhong, Mingming Li, Li Zhou, Wei Shen, Govender Algasan, You Fan, and Zhengxiang Wang. “Metabolic Engineering of Escherichia Coli for Improving Shikimate Synthesis from Glucose.” Bioresource Technology 166 (August 1, 2014): 64–71. https://doi.org/10.1016/j.biortech.2014.05.035.
